# COGS 188 - Project Proposal

# Names

- Sebastian Modafferi
- Chirag Amatya
- Michael Chu

# Abstract 

The goal of this project is to develop a reinforcement learning agent capable of successfully landing a spacecraft in the Lunar Lander environment provided by Gymnasium. This environment simulates a classic rocket trajectory optimization problem, controlling of the lander’s engines to achieve a safe landing. The RL agent will use the 8-dimensional state space, consisting of position, velocity, orientation, and ground contact indicators, to select discrete actions such as firing the main or orientation engines. A reward-based feedback system will guide the agent, encouraging successful landings while penalizing crashes and excessive fuel usage. The RL agent will process the data to learn an optimal policy for controlling the spacecraft through iterative training. Success will be measured using metrics such as average total reward per episode, success rate (percentage of safe landings), and fuel efficiency.

# Background

Reinforcement learning (RL) has become a prominent approach for solving complex decision-making tasks, particularly in environments requiring continuous control and sequential decision-making. The Lunar Lander task, provided by the Gymnasium framework, serves as an effective benchmark for evaluating RL algorithms. This task involves navigating a simulated spacecraft to a safe landing position using thrusters while minimizing fuel consumption. Given the large, continuous state space of the environment, traditional dynamic programming methods are infeasible as they require complete knowledge of the environment's state transitions <a name="suttonnote"></a>[1]. Instead, Monte Carlo (MC) methods, which learn directly from episodic experiences, provide a more suitable alternative, though their reliance on complete episodes for updates results in slower convergence and increased memory demands <a name="2"></a>[2] .

Temporal-Difference (TD) learning methods offer a solution to these limitations by updating value estimates at each step rather than waiting for entire episodes to complete. Q-learning, a widely used TD method, learns an optimal policy by iteratively improving state-action values without requiring a full model of the environment <a name="3"></a>[3] 
. However, in environments with high-dimensional state spaces, tabular representations of Q-values become impractical, necessitating function approximation techniques. Deep Q-Networks (DQN), introduced by DeepMind, leverage deep neural networks to approximate Q-values, enabling learning in complex, high-dimensional environments <a name="4"></a>[4] 
. This breakthrough demonstrated that deep reinforcement learning could achieve human-level performance in various control tasks, such as playing Atari games.

In this project, we explore the use of DQN and Double DQN (DDQN) to optimize landing strategies in the Lunar Lander environment. By evaluating their performance in terms of convergence speed, stability, and policy effectiveness, we aim to gain insights into the effectiveness of deep reinforcement learning in real-world-inspired autonomous control tasks.

# Problem Statement

The problem we are solving is the [lunar lander task](https://gymnasium.farama.org/environments/box2d/lunar_lander/) provided by the `gymnasium` python package. The task is to land a vehicle powered by three engines onto a randomly-generated surface. The lander is comrpised of two orientation engines, which add angular velocity to rotate the lander, along with one main engine which provides x/y-velocity depending on the orientation. The task begins with the lander in the top-center of the simulation, with a random initial force acting upon it. The lander must then use the three engines to not only correct for the initial force, but to also touchdown near (0,0), such that the two "legs" contact the surface. In order to successfully complete the task the lander must land close to (0,0) and must use the engines sparingly, as their is a reward cost for engine usage. The total reward breakdown is listed below, as described on [the docs](https://gymnasium.farama.org/environments/box2d/lunar_lander/).


- **Increases** the closer the lander is to the landing pad and **decreases** the further it is.
- **Increases** the slower the lander is moving and **decreases** the faster it is moving.
- **Decreases** the more the lander is tilted (i.e., the angle is not horizontal).
- **Increases** by **10 points** for each leg that is in contact with the ground.
- **Decreases** by **0.03 points** for each frame a side engine is firing.
- **Decreases** by **0.3 points** for each frame the main engine is firing.

Additionally, the episode receives:

- **-100 points** for crashing.
- **+100 points** for landing safely.

In order to successfully complete the task, the lander must achieve __at least__ 200 points. Additional points will be used to compare performance across multiple agents.

# Data

For the Lunar Lander environment from Gymnasium, the data consists of simulated episodes of landing attempts. The dataset is dynamically generated using the environment and represents the lander’s state transitions as it interacts with the environment. Due to the nature of training reinforcement learning models, no dataset will be collected or manufactured from the simulation Instead, the agents will learn via exploration and repeated simulations.

- **Source**: Gymnasium Lunar Lander environment (`gymnasium.make("LunarLander-v3")`).
- **Size**: The exact number of episodes required for training will vary by model, however we are looking to benchmark progress after 10,000 episodes of training. This benchmark will then compute the average performance over 100 simulations.
- **Observation Space**:
  - An 8-dimensional state vector:
    1. Horizontal position ($x$): Range $[-2.5, 2.5]$
    2. Vertical position ($y$): Range $[-2.5, 2.5]$
    3. Horizontal velocity ($v_x$): Range $[-10, 10]$
    4. Vertical velocity ($v_y$): Range $[-10, 10]$
    5. Angle ($\theta$): Range $[-6.283, 6.283]$ radians
    6. Angular velocity ($\omega$): Range $[-10, 10]$
    7. Left leg ground contact ($boolean$): 0 (not in contact) or 1 (in contact)
    8. Right leg ground contact ($boolean$): 0 (not in contact) or 1 (in contact)
  - The observation values are floating-point numbers ($float32$).

- **Action Space**: 
  - A set of 4 possible actions:
    1. 0: Do nothing
    2. 1: Fire the left orientation engine
    3. 2: Fire the main engine
    4. 3: Fire the right orientation engine

- **Reward Function**:
  - Positive reward for proximity to the landing pad and stable velocity.
  - $+10$ points for each leg in contact with the ground.
  - $-0.03$ points per frame for using side thrusters.
  - $-0.3$ points per frame for using the main engine.
  - Additional $+100$ points for a successful landing.
  - Additional $-100$ points for a crash.

- **Starting State**:
  - Lander starts at the top center of the viewport with randomized initial force applied to its center of mass.

#### Data Handling (WIP/Not Final):
- **Transformations**:
  - Normalize continuous state variables to the range $[0, 1]$ for better convergence.
  - One-hot encode discrete actions for compatibility with certain ML models

# Proposed Solution

Three agents will be trained in order to compare performance across different model architypes. They will vary in complexity, in an attempt to gauge the required complexity for our task. All agents will be compared across our evaluation metrics.

(Chirag) The first agent will be Q-learning based. Q-learning is a run-of-the-mill approach for any reinforcement learning task, and serves as a strong benchmark for performance. It is commonly used in agent-based environments due to it's proficiency with exploration. With the observation space being continuous, the Q-table will be comprised of bins which will serve to discretize the observation space. This is an efficient approach since while the observations are continuous, they do fall within a set range (see Data section). The exact number of bins per observation variable will be hyper-parameter optimized for, however we will begin with 100 bins per variable as a preliminary model. 

(Micheal) The second agent will be trained through a Soft Actor-Critic discrete (SAC-Discrete) implementation. SAC agents are designed to handle continuous action spaces, however through the SAC-Discrete implementation, we are able to use this approach. SAC-Discrete is an efficient approach due to the inclusion of an entropy term to the objective function. This entropy term will encourage the model to explore and learn, which will be helpful with inital learning. Due to the nature of the task involving an initially random velocity, it is important for the model to explore which different methods of stabilizing the lander are efficient given task rewards. The exact value of this entropy coefficient will be hyper-parameter optimized for in an attempt to both reduce training time and increase final performance.

(Sebastian) The third agent will be a genetically-trained neural network (inspiration<a name="gtnn"></a>[<sup>[5]</sup>](#gtnn)). The exact nueral network structure will remain consisent across generations. However the weights, biases, and connections will be trained through mating and mutating the highest performing members of the population. The exact neuron and layer counts will be hyperparameter optimized for. This will be an effective solution because neural networks are well suited for agent-based tasks, specifically where the agent has a discrete set of actions to take in an environment. Through many generations of breeding and mutating the highest performing genomes, the agent will increasingly perform better and better. Genome-based learning is essential for this implementation due to the lack of a dataset. Using a reinforecement-learning approach will remove the need for us to manufacture a dataset from the environment -- instead allowing the agent to explore the action-observation space relationship.

# Evaluation Metrics

To evaluate the performance of the model, the following metrics will be used:

#### Average Total Reward Per Episode
- Measures the cumulative reward collected over an episode.
- Higher rewards indicate better lander behavior.
- **Formula**:

  $R_{\text{total}} = \sum_{t=1}^{T} r_t$

  where $r_t$ is the reward at timestep $t$, and $T$ is the total number of timesteps.
- This metric will be used during model training and will be the primary mode of evaluation.

#### Success Rate
- Percentage of episodes where the lander safely lands on the designated landing pad.
- **Formula**:

  $\text{Success Rate} = \frac{\text{Number of Successful Landings}}{\text{Total Episodes}} \times 100$

#### Fuel Efficiency
- Average fuel usage across episodes. Lower values indicate more efficient lander behavior.
- **Formula**:

  $\text{Fuel Efficiency} = \frac{\text{Total Fuel Consumed}}{\text{Number of Episodes}}$

# Ethics & Privacy

Ethics and privacy concerns in the Lunar Lander project primarily revolve around reinforcement learning risks, data handling, and potential real-world implications. Since the model learns through trial and error, there is a risk of unsafe exploration if not properly controlled, which could encourage reckless decision-making in real-world applications. Additionally, while this project operates in a simulated environment, lessons from it could influence autonomous landing systems, requiring careful consideration of safety and bias. From a data perspective, ensuring fair training practices, avoiding unintended biases, and minimizing environmental impact from high computational resource usage are key concerns. Implementing fairness-aware learning, optimizing resource use, and maintaining transparency in model behavior will help mitigate these risks.

# Team Expectations 

Put things here that cement how you will interact/communicate as a team, how you will handle conflict and difficulty, how you will handle making decisions and setting goals/schedule, how much work you expect from each other, how you will handle deadlines, etc...
 - Communicate delays or updates on progress as soon as possible.
 - Attend meetings on time - or notify of delays prior.
 - Be respectful of other's opinions.
 - Complete your assigned work with integrity and to your best ability.
 - Share bugs that could be common across other tasks - improve eachother's ability to work efficiently.

# Project Timeline Proposal

Replace this with something meaningful that is appropriate for your needs. It doesn't have to be something that fits this format.  It doesn't have to be set in stone... "no battle plan survives contact with the enemy". But you need a battle plan nonetheless, and you need to keep it updated so you understand what you are trying to accomplish, who's responsible for what, and what the expected due dates are for each item.

| Meeting Date  | Meeting Time| Completed Before Meeting  | Discuss at Meeting |
|---|---|---|---|
| 2/11  |  2 PM | Brainstorm topics/questions (all)  | Communication methods - Project Task - Separation of work for Proposal | 
| 2/13  |  6 PM | Completed proposal assigned sections | discuss proposal submission |
| 2/18  |  2 PM | Decide personal preferences for different approaches | Assign each group member to an approach - Finalize timeline | 
| 2/25  | 2 PM  | boilerplate for training code complete | Discuss goals with model training |
| 2/14  | 2 PM  | First agents should be complete | Discuss hyper-parameter optimization across different approaches |
| 2/23  | 2 PM  | Final agents complete | Discuss code tyding, exporting large functions to .py files for a more readable notebook. |
| 3/4  | 2 PM  | Completion for code-side of Project | Discuss writing strategies - compare preformances |
| 3/11 | 2 PM | Completion of final project submission | Go over the final submission as a group, prepare to submit. |
| 3/19  | Before 11:59 PM  | Whole Project! | Turn in Final Project  |

# Footnotes
1. Sutton, R. S., & Barto, A. G. (2018). Reinforcement learning: An introduction (2nd ed.). MIT Press.

2. Watkins, C. J. C. H., & Dayan, P. (1992). Q-learning. Machine Learning, 8(3-4), 279-292. https://doi.org/10.1007/BF00992698 

3. Mnih, V., Kavukcuoglu, K., Silver, D., Rusu, A. A., Veness, J., Bellemare, M. G., Graves, A., Riedmiller, M., Fidjeland, A. K., Ostrovski, G., Petersen, S., Beattie, C., Sadik, A., Antonoglou, I., King, H., Kumaran, D., Wierstra, D., Legg, S., & Hassabis, D. (2015). Human-level control through deep reinforcement learning. Nature, 518(7540), 529-533. https://doi.org/10.1038/nature14236 

4. Russell, S., & Norvig, P. (2020). Artificial Intelligence: A Modern Approach (4th ed.). Pearson.

5. Training a Neural Network using Genetic Algorithm (22 May 2020) [Pezzza's Work](https://www.youtube.com/watch?v=I_p-yfdyP1A).